In [2]:
from concurrent import futures


In [3]:
from mail_merge import *

In [4]:
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
import subprocess
from docx.shared import Pt
import pandas as pd
import os
import shutil
from tqdm import tqdm

In [10]:
shutil.rmtree('output')
os.mkdir('output')
df_input = pd.read_excel('Input/sample_consult_file_w_prescription.xlsx')
df_input['ICDX'] = df_input.filter(like = 'ICD').astype(str).agg(','.join, axis=1).str.replace(',nan', '')
df_input['Diagnosis'] = df_input.filter(like = 'Diagnosis ').astype(str).agg(','.join, axis=1).str.replace(',nan', '')
max_pres = df_input.filter(like = 'obat_').columns[-1].split('_')[1]

df_input['pres_all'] = ''
for i in range(int(max_pres)):
    j = i + 1
    new_pres = df_input[[x +'_'+ str(j) for x in ['obat', 'harga', 'jumlah', 'total']]].astype(str).agg(';'.join, axis=1)
    df_input['pres_all'] = df_input['pres_all'] + '|' + new_pres
# for i in 
# df_input.filter(like = 'obat_')
df_input['pres_all'] = df_input['pres_all'].str.replace('^.', '', regex = True)
df_input['pres_all']  = df_input['pres_all'].str.replace('nan;', '').str.replace('nan|', '').str.replace('|nan', '')

df_input['Rx Fee'] = df_input['Rx Fee'].fillna(0)
df_input['Consult Fee'] = df_input['Consult Fee'].fillna(0)


In [11]:
for index, row in tqdm(df_input.iterrows()):
    row_to_pdf(row)

1000it [01:09, 14.37it/s]


In [16]:
import glob
for i in tqdm(glob.glob('output/*')):
     subprocess.run(['libreoffice', '--convert-to', 'pdf' ,
                    i, '--outdir', 'output/']
                   ,stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT
                )

100%|██████████| 963/963 [34:55<00:00,  2.18s/it]


In [9]:
df_input.shape

(1000, 129)

In [7]:
import concurrent

In [8]:
%%time
executor = concurrent.futures.ProcessPoolExecutor(8)
futures = [executor.submit(row_to_pdf, item) for item in df_input.to_dict(orient='records')]
concurrent.futures.wait(futures)

CPU times: user 666 ms, sys: 211 ms, total: 876 ms
Wall time: 23.7 s


DoneAndNotDoneFutures(done={<Future at 0x7fb1c7bf0050 state=finished returned NoneType>, <Future at 0x7fb1c7968080 state=finished returned NoneType>, <Future at 0x7fb1c7988080 state=finished returned NoneType>, <Future at 0x7fb1c7bf00e0 state=finished returned NoneType>, <Future at 0x7fb1c79300e0 state=finished returned NoneType>, <Future at 0x7fb1c79d00e0 state=finished returned NoneType>, <Future at 0x7fb1c79e00e0 state=finished returned NoneType>, <Future at 0x7fb1c79f00e0 state=finished returned NoneType>, <Future at 0x7fb1c7968170 state=finished returned NoneType>, <Future at 0x7fb1c7988170 state=finished returned NoneType>, <Future at 0x7fb1c79301a0 state=finished returned NoneType>, <Future at 0x7fb1c7bf01d0 state=finished returned NoneType>, <Future at 0x7fb1c79d01d0 state=finished returned NoneType>, <Future at 0x7fb1c79e01d0 state=finished returned NoneType>, <Future at 0x7fb1c79f01d0 state=finished returned NoneType>, <Future at 0x7fb1c7930260 state=finished returned NoneTyp